In [ ]:
# import mne
# import numpy as np
# from mne.datasets import eegbci
# import matplotlib.pyplot as plt
# import pywt
# from mne.channels import make_standard_montage

# fs = 250.0
 
# channels = ["Fz","C3", "Cz", "C4","Pz","PO7","Oz","PO8"]

# info = mne.create_info(
#     ch_names= channels,
#     ch_types= ['eeg']*len(channels),
#     sfreq= fs
# )

# channels2 = ["Fz","C3", "Cz", "C4","Pz","PO7","PO8"]

# info2 = mne.create_info(
#     ch_names= channels2,
#     ch_types= ['eeg']*len(channels2),
#     sfreq= fs
# )

# def call_pywt(new_data):
#     component_num = 7
#     train_cwt = np.ndarray(shape=(new_data.shape[0], new_data.shape[2], component_num))
#     for jj in range(0, new_data.shape[0]):
#         train_cwt[jj] = new_data[jj].T

#     scales = range(1,31)
#     waveletname = 'morl'
#     train_size = new_data.shape[0]
#     train_data_cwt = np.ndarray(shape=(train_size, len(scales), new_data.shape[2], component_num))

#     for ii in range(0,train_size):
#         for jj in range(0,component_num):
#             signal = train_cwt[ii, :, jj]
#             coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
#             coeff_ = coeff[:,:new_data.shape[2]]  
#             train_data_cwt[ii, :, :, jj] = coeff_

#     train_cwt_stack = np.ndarray(shape=(train_size, len(scales)*component_num, new_data.shape[2]))

#     for jj in range(0,train_data_cwt.shape[0]):
#         train_cwt_stack[jj] = np.vstack((train_data_cwt[jj,:,:,0], train_data_cwt[jj,:,:,1], train_data_cwt[jj,:,:,2], train_data_cwt[jj,:,:,3], train_data_cwt[jj,:,:,4], train_data_cwt[jj,:,:,5], train_data_cwt[jj,:,:,6]))
#     return train_cwt_stack

# raw_edf = mne.io.read_raw_edf("C:\git\Senior_Thesis\DataSet\Convert_data\hand3_new.edf",preload = True)

# # epoch_2s = raw_edf.get_data()[:,0:501]

# # epoch_2s = raw_edf.get_data()[:,501:1002]

# epoch_2s = raw_edf.get_data()[:,6251:6251+501]

# raw = mne.io.RawArray(epoch_2s, info, verbose=False)
# # eeg1 = raw.copy().filter(l_freq=6.0, h_freq=32.0, method = 'iir', iir_params= {"order": 6, "ftype":'butter'}, verbose = False)

# eeg1 = raw.copy().set_eeg_reference(ref_channels="average", verbose=False)
# eeg1= eeg1.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','PO8'])

# mne_epoch = mne.EpochsArray(eeg1.get_data().reshape(1,7,501), info2, verbose=False)
# mne_epoch = mne_epoch.copy().filter(l_freq=6.0, h_freq=32.0, method = 'iir', iir_params= {"order": 6, "ftype":'butter'}, verbose = False)


Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand3_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...


In [ ]:
# print(mne_epoch.get_data().max())
# mne_epoch.get_data()

174.25573710792204


array([[[ 3.04417407e-02, -4.01785091e+01, -6.89950216e+01, ...,
          1.54904150e+01,  9.14725578e+00, -1.12853510e-02],
        [ 2.20284411e-03, -5.91018300e+01, -1.01286482e+02, ...,
         -2.85077833e+01, -1.67013940e+01, -4.93309491e-03],
        [-5.33353239e-02, -6.82498883e+01, -1.16101327e+02, ...,
         -2.15299576e+02, -1.26301519e+02, -2.07051474e-03],
        ...,
        [ 2.50817999e-02,  1.24964778e+01,  2.11220175e+01, ...,
          1.13040730e+02,  6.60562141e+01,  3.39453965e-03],
        [-2.55959170e-04,  8.92712792e+01,  1.52302782e+02, ...,
          7.69646039e+01,  4.53493981e+01,  4.01603561e-03],
        [ 1.82506762e-02,  1.70808795e+01,  2.95502662e+01, ...,
          9.37446813e+01,  5.51052605e+01,  8.92423362e-03]]])

# CNN model

In [2]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import mne
import pywt
import pickle
from keras.models import load_model
import pandas as pd
from datetime import datetime
import os
from sklearn.preprocessing import StandardScaler

pretrained_model = load_model("RNN_model_newsess_nonvsim_weights.h5")

epoch_2s = np.zeros(shape=(8,501))

# first resolve an EEG stream on the lab network
print("looking for an EEG stream...")
All_streams = resolve_stream()
print(All_streams)

for i in All_streams:
    print(i.name())

unicorn_stream =[stream for stream in All_streams if stream.name() == 'openvibefilter']

data = []
time_count = 0.0
time_all = 0.0
time_interval = 0.1

inlet = StreamInlet(unicorn_stream[0])

fs = 250.0
 
channels = ["Fz","C3", "Cz", "C4","Pz","PO7","Oz","PO8"]

info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= fs
)

channels2 = ["Fz","C3", "Cz", "C4","Pz","PO7","PO8"]

info2 = mne.create_info(
    ch_names= channels2,
    ch_types= ['eeg']*len(channels2),
    sfreq= fs
)

# Example usage
csv_file = 'real_time_data.csv'

if os.path.exists(csv_file):
    open(csv_file, 'w').close()


def call_pywt(new_data):
    component_num = 8
    train_cwt = np.ndarray(shape=(new_data.shape[0], new_data.shape[2], component_num))
    for jj in range(0, new_data.shape[0]):
        train_cwt[jj] = new_data[jj].T

    scales = range(1,31)
    waveletname = 'morl'
    train_size = new_data.shape[0]
    train_data_cwt = np.ndarray(shape=(train_size, len(scales), new_data.shape[2], component_num))

    for ii in range(0,train_size):
        for jj in range(0,component_num):
            signal = train_cwt[ii, :, jj]
            coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
            coeff_ = coeff[:,:new_data.shape[2]]  
            train_data_cwt[ii, :, :, jj] = coeff_

    train_cwt_stack = np.ndarray(shape=(train_size, len(scales)*component_num, new_data.shape[2]))

    for jj in range(0,train_data_cwt.shape[0]):
        train_cwt_stack[jj] = np.vstack((train_data_cwt[jj,:,:,0], train_data_cwt[jj,:,:,1], train_data_cwt[jj,:,:,2], train_data_cwt[jj,:,:,3], train_data_cwt[jj,:,:,4], train_data_cwt[jj,:,:,5], train_data_cwt[jj,:,:,6],train_data_cwt[jj,:,:,7] ))
    return train_cwt_stack

count = 0

while True:

    sample, timestamp = inlet.pull_sample()
    if sample:

        count += 1

        data1 = sample[0:8]
        # data1.append(sample[7])
        data.append(data1)  

        time_count += 1 * 0.004
        time_count = np.round(time_count ,3)
        time_all += 1 * 0.004

        if time_count == time_interval:

            data = np.asarray(data)
            epoch_2s = np.delete(epoch_2s, slice(0,int(time_interval/0.004)), axis =1)
            epoch_2s = np.append(epoch_2s, data.T, axis= 1)

            # print(epoch_2s)

            # data = np.asarray(data)
            # data = np.transpose(data)
            # epoch_2s = data
            # print(epoch_2s)
            
            raw = mne.io.RawArray(epoch_2s, info, verbose=False)
            
            # eeg1 = raw.copy().set_eeg_reference(ref_channels="average", verbose=False)
            eeg1= raw.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','Oz','PO8'])

            mne_epoch = mne.EpochsArray(eeg1.get_data().reshape(1,8,501), info, verbose=False)
            # mne_epoch = mne_epoch.copy().filter(l_freq=6.0, h_freq=32.0, method = 'iir', iir_params= {"order": 6, "ftype":'butter'}, verbose = False)

            # print("*****************Fitered*******************")
            # print(mne_epoch.get_data())
            # print(mne_epoch.get_data().max())

            with open('trained_csp_newsess_nonvsim_model.pkl', 'rb') as file:
                trained_csp = pickle.load(file)

            new_data = trained_csp.transform(mne_epoch.get_data().reshape(1,8,501))

            # train_cwt_stack = call_pywt(new_data)

            # scaler = StandardScaler()
            # reshaped_data = train_cwt_stack.reshape((train_cwt_stack.shape[0], -1))
            # scaled_data = scaler.fit_transform(reshaped_data)
            # scaled_data_3d = scaled_data.reshape(train_cwt_stack.shape)
            # print(scaled_data_3d.shape)


            y_pred_raw = pretrained_model.predict(new_data ,verbose=False)

            # print("************************************")
            # print(y_pred_raw)
            print("predict class", np.argmax(y_pred_raw))

            new_data_point = {
            'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Class_Raw': y_pred_raw[0],
            'Class': np.argmax(y_pred_raw)
            }

            # Create a DataFrame from the new_data_point
            df = pd.DataFrame([new_data_point])
            try:
                df.to_csv(csv_file, mode='x', index=False)
            except FileExistsError:
                df.to_csv(csv_file, mode='a', header=False, index=False)

                    
            data = np.ndarray.tolist(data)
            data = []

            time_count = 0.0
        
        # if time_all >= 150.0:
        #     break

looking for an EEG stream...
[<pylsl.pylsl.StreamInfo object at 0x0000019F541E3450>, <pylsl.pylsl.StreamInfo object at 0x0000019F5D0BDBD0>, <pylsl.pylsl.StreamInfo object at 0x0000019F68A85A50>]
openvibefilter
openvibeMarker
UN-2019.08.06
predict class 1
predict class 1
predict class 0
predict class 1
predict class 1
predict class 0
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 0
predict class 1
predict class 1
predict class 1
predict class 0
predict class 0
predict class 0
predict class 0
predict class 1
predict class 0
predict class 1
predict class 0
predict class 0
predict class 1
predict class 0
predict class 1
predict class 1
predict class 0
predict class 1
predict class 0
predict class 1
predict class 0
predict class 0
predict class 0
predict class 0
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict class 1
predict c

# CSP+LDA

In [1]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib.animation import FuncAnimation
import os
from datetime import datetime
import pandas as pd

epoch_2s = np.zeros(shape=(8,501))

# first resolve an EEG stream on the lab network
print("looking for an EEG stream...")
All_streams = resolve_stream()
print(All_streams)

for i in All_streams:
    print(i.name())

unicorn_stream =[stream for stream in All_streams if stream.name() == 'openvibefilter']

data = []
time_count = 0.0
time_all = 0.0
time_interval = 0.1

inlet = StreamInlet(unicorn_stream[0])

fs = 250.0
 
channels = ["Fz","C3", "Cz", "C4","Pz","PO7","Oz","PO8"]

info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= fs
)

channels2 = ["Fz","C3", "Cz", "C4","Pz","PO7","PO8"]

info2 = mne.create_info(
    ch_names= channels2,
    ch_types= ['eeg']*len(channels2),
    sfreq= fs
)

csv_file = 'real_time_data.csv'

if os.path.exists(csv_file):
    open(csv_file, 'w').close()


while True:
    sample, timestamp = inlet.pull_sample()
    # print(sample)

    if sample:
        data1 = sample[0:8]
        data.append(data1)  

        time_count += 1 * 0.004
        time_count = np.round(time_count ,3)
        time_all += 1 * 0.004

        if time_count == time_interval:

            data = np.asarray(data)
            epoch_2s = np.delete(epoch_2s, slice(0,int(time_interval/0.004)), axis =1)
            epoch_2s = np.append(epoch_2s, data.T, axis= 1)
            

            raw = mne.io.RawArray(epoch_2s, info, verbose=False)
            
            # eeg1 = raw.copy().set_eeg_reference(ref_channels="average", verbose=False)
            eeg1= raw.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','Oz','PO8'])

            mne_epoch = mne.EpochsArray(eeg1.get_data().reshape(1,8,501), info, verbose=False)

            with open('trained_csp2_model.pkl', 'rb') as file:
                trained_csp2 = pickle.load(file)
            new_data2 = trained_csp2.transform(mne_epoch.get_data().reshape(1,8,501))


            with open('trained_lda_model.pkl', 'rb') as file:
                trained_lda = pickle.load(file)

            # print(trained_lda.predict_proba(new_data2))

            print(trained_lda.predict(new_data2))

            # if trained_lda.predict(new_data2)[0] == 0:
            #     print('class predict LEFT')
            # # elif trained_lda.predict(new_data2)[0] == 1:
            # #     print('class predict RIGHT')
            # elif trained_lda.predict(new_data2)[0] == 1:
            #     print('class predict NON-MI')
            # else:
            #     print('class predict UP')


            new_data_point = {
            'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Class_Raw': trained_lda.predict_proba(new_data2),
            'Class': trained_lda.predict(new_data2)
            }

            # Create a DataFrame from the new_data_point
            df = pd.DataFrame([new_data_point])
            try:
                df.to_csv(csv_file, mode='x', index=False)
            except FileExistsError:
                df.to_csv(csv_file, mode='a', header=False, index=False)
        
            data = np.ndarray.tolist(data)
            data = []
            time_count = 0.0


looking for an EEG stream...
[<pylsl.pylsl.StreamInfo object at 0x000002962EEA0ED0>, <pylsl.pylsl.StreamInfo object at 0x000002962C266B90>, <pylsl.pylsl.StreamInfo object at 0x000002962F414E50>]
openvibefilter
openvibeMarker
UN-2019.08.06
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[1]
[1]
[0]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[